## Collecting protocol fees across Balancer core pools on all networks
Spreadsheet as reference: https://docs.google.com/spreadsheets/d/1xwUPpbYq7woVOU9vQ8EB8MY75I-1mauTLyDVwvKUDKo/edit#gid=0
Collab: https://colab.research.google.com/drive/1vKCvcV5mkL1zwW3565kLSGkBEbt8NsoB?usp=sharing


In [60]:
# Define constants for Arbitrum:
ARB_CORE_POOLS = ["RDNT-WETH", "rETH-bb-a-WETH-BPT", "50wstETH-50WETH", "50wstETH-BPT-50bbaUSD", "wstETH/rETH/cbETH", "50USDC-50USDC.e"]
ARB_BALANCER_GRAPH_URL = "https://api.thegraph.com/subgraphs/name/balancer-labs/balancer-arbitrum-v2"
# Arb block time is .3 seconds
ARB_CHAIN_BLOCK_TIME = 0.27

In [61]:
# Query:
POOLS_SNAPSHOTS_QUERY = """
{{
  poolSnapshots(
    first: {first}
    skip: {skip}
    orderBy: timestamp
    orderDirection: desc
    block: {{ number: {block} }}
    where: {{ protocolFee_not: null }}
  ) {{
    pool {{
      address
      id
      symbol
    }}
    timestamp
    protocolFee
    swapFees
    swapVolume
    liquidity
  }}
}}
"""

## Fetching data from the Balancer subgraphs

In [62]:
import os
import datetime
from typing import Dict
from typing import List
from typing import Optional

from dotenv import load_dotenv
from gql import Client
from gql import gql
from gql.transport.requests import RequestsHTTPTransport
from web3 import Web3

load_dotenv()
arb_web3 = Web3(Web3.HTTPProvider(os.environ["ARBNODEURL"]))

arb_block_now = arb_web3.eth.block_number - 1000
arb_timestamp_now = arb_web3.eth.get_block(arb_block_now).timestamp
# Given Arb block time, we want to look back 2 weeks:
block_2_weeks_ago = arb_block_now - (2 * 7 * 24 * 60 * 60 / ARB_CHAIN_BLOCK_TIME)
arb_timestamp_2_weeks_ago = arb_web3.eth.get_block(int(block_2_weeks_ago)).timestamp

# Convert to datetime:
arb_datetime_now = datetime.datetime.fromtimestamp(arb_timestamp_now)
arb_datetime_2_weeks_ago = datetime.datetime.fromtimestamp(arb_timestamp_2_weeks_ago)

# Fetch all the data from the balancer subgraph
def make_gql_client(url: str) -> Optional[Client]:
    transport = RequestsHTTPTransport(url=url, retries=3)
    return Client(
        transport=transport, fetch_schema_from_transport=True, execute_timeout=60
    )


def get_balancer_pool_snapshots(block: int, graph_url: str) -> Optional[List[Dict]]:
    client = make_gql_client(graph_url)
    all_pools = []
    limit = 1000
    offset = 0
    while True:
        result = client.execute(
            gql(POOLS_SNAPSHOTS_QUERY.format(first=limit, skip=offset, block=block)))
        all_pools.extend(result['poolSnapshots'])
        offset += limit
        if offset >= 5000:
            break
        if len(result['poolSnapshots']) < limit - 1:
            break
    return all_pools


pool_snapshots_now = get_balancer_pool_snapshots(arb_block_now, ARB_BALANCER_GRAPH_URL)
pool_snapshots_2_weeks_ago = get_balancer_pool_snapshots(int(block_2_weeks_ago), ARB_BALANCER_GRAPH_URL)

## Extract fee data from CORE pools:


In [63]:
import pandas as pd
# Iterate through snapshots now and 2 weeks ago and extract fee data, by subtracting today's fee data from 2 weeks ago
# and then summing across all pools
arb_fees = {}
for pool in ARB_CORE_POOLS:
    current_fees_snapshots = [x for x in pool_snapshots_now if x['pool']['symbol'] == pool]
    fees_2_weeks_ago = [x for x in pool_snapshots_2_weeks_ago if x['pool']['symbol'] == pool]
    # Take first element of list, which is the most recent snapshot
    pool_snapshot_now = current_fees_snapshots[0]
    pool_snapshot_2_weeks_ago = fees_2_weeks_ago[0]
    # Calculate fees
    pool_fee = float(pool_snapshot_now['protocolFee']) - float(pool_snapshot_2_weeks_ago['protocolFee'])
    arb_fees[pool] = {
        'pool_fee': pool_fee, 'time_from': arb_datetime_2_weeks_ago, 'time_to': arb_datetime_now,'pool_symbol': pool_snapshot_now['pool']['symbol'],
        'chain': 'Arbitrum',
    }
# Convert to dataframe and print:
arb_fees_df = pd.DataFrame.from_dict(arb_fees, orient='index')
arb_fees_df

,pool_fee,time_from,time_to,pool_symbol,chain
RDNT-WETH,12414.282233,2023-07-28 21:39:13,2023-08-11 14:49:31,RDNT-WETH,Arbitrum
rETH-bb-a-WETH-BPT,0.000000,2023-07-28 21:39:13,2023-08-11 14:49:31,rETH-bb-a-WETH-BPT,Arbitrum
50wstETH-50WETH,2.018571,2023-07-28 21:39:13,2023-08-11 14:49:31,50wstETH-50WETH,Arbitrum
50wstETH-BPT-50bbaUSD,1.033910,2023-07-28 21:39:13,2023-08-11 14:49:31,50wstETH-BPT-50bbaUSD,Arbitrum
wstETH/rETH/cbETH,21.035821,2023-07-28 21:39:13,2023-08-11 14:49:31,wstETH/rETH/cbETH,Arbitrum
50USDC-50USDC.e,38.525948,2023-07-28 21:39:13,2023-08-11 14:49:31,50USDC-50USDC.e,Arbitrum
